Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0.01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos o valor zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [1]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



In [5]:
# CONSTANTES

# relacionadas à busca
TAMANHO_POP = 30
NUM_GERACOES = 50
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 10
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "Notebook gamer": {
        "peso": 3,
        "valor": 10000,
    },
    "Trilogia de senhor dos anéis": {
        "peso": 1,
        "valor": 500,
    },
    "Torradeira": {
        "peso": 2,
        "valor": 200,
    },
    "Microscópio ótico": {
        "peso": 5,
        "valor": 3000,
    },
    "Binóculos": {
        "peso": 0.5,
        "valor": 1000,
    },
    "Telescópio": {
        "peso": 2,
        "valor": 7000,
    },
    "Coletânea das sinfonias de Beethoven em vinil": {
        "peso": 1,
        "valor": 2000,
    },
    "vitrola": {
        "peso": 7,
        "valor": 3000,
    },
    "Máquina de escrever": {
        "peso": 3,
        "valor": 4000,
    },
    "Lego da Pirâmide de Gizé": {
        "peso": 1.5,
        "valor": 1000,
    },
}

NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES)

In [11]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variáveis para o hall da fama

melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_OBJETOS

for n in range(NUM_GERACOES):

    # Seleção
    
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness) # associa cada elemento na lista ao objeto no dicionário
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado

print()
print('Você deve pegar os seguintes itens:')
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print('+', item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f'A mochila tem o valor de {valor_total} dinheiros'
    f' e peso de {peso_total} unidades de massa'
)

Maior valor: 20700 | Peso: 9.5

Você deve pegar os seguintes itens:
+ Coletânea das sinfonias de Beethoven em vinil
+ Máquina de escrever
+ Notebook gamer
+ Telescópio
+ Trilogia de senhor dos anéis

A mochila tem o valor de 23500 dinheiros e peso de 10 unidades de massa


## Conclusão



Esse experimento consiste em um problema do tipo NP difícil de maximização, no qual uma mochila deve comportar o máximo de objetos possíveis de maior valor total e que não esgote o peso que a mochila pode suportar. A melhor resposta seria obtida se fosse possível testar todas as probabilidades, porém devido a exigência de bastante recurso computacional, extrapolando a capacidade da máquina em executar a solução ideal para esse tipo de problema, o algoritmo testará uma lista de indivíduos na população para obter o melhor resultado.

Para esse problema foi preciso criar um dicionário de dicionários, no qual cada objeto é um dicionário que apresenta duas chaves (peso e valor) atribuído à ele, para conseguir extrair as informações necessárias, a fim de encontrar os objetos que cabem na mochila sem exceder sua capacidade limite e que apresentem na somatória o maior valor possível dentre as possibilidades.

O algoritmo utiliza a mesma estrutura dos experimentos anteriores para as funções gene e indivíduo, pois geram indivíduos aleatórios que também passam pelo processo de seleção, cruzamento e mutação, com o objetivo de melhorar as possibilidades para os indivíduos dentro da lista inicial que foi estabelecida ao acaso.

A estratégia para a solução desse problema foi embasada na caixa binária, em que na lista binária [0, 1] o 1 representa o objeto que está na mochila e o 0 é o objeto que não foi considerado.

Foi definida a função computa mochila que é uma função intermediária de suporte para a função objetivo, e serve para calcular o valor e peso totais da mochila, facilitando o processo de identificar o melhor fitness na função objetivo aplicada a cada indivíduo para selecionar os melhores.

Esse problema pode ser aplicado em uma molécula de fulereno que funciona como *drug delivery*. A molécula é a mochila e cada componente molecular é um objeto que tem uma função no organismo humano, sendo que sua combinação apresenta maior ou menor eficiência (valor) e um volume específico (peso) que pode ocupar o interior do fulereno, cujo espaço é limitado pela sua nanoestrutura icosaédrica.

## Playground

